In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import sys
import time
import gc
import itertools
import tqdm
import time
import copy

import scipy.stats as ss
from numba import jit

if os.name=='nt':
    try:
        mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
        os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
    except:
        pass
    
import xgboost as xgb
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

%matplotlib inline

### Helper Function

In [2]:
def lag_features(df, lags, merge_cols, shift_cols, fillna_value=None):
    '''create lag features of col'''
    cols = copy.copy(merge_cols)
    cols.extend(shift_cols)
    tmp = df.loc[:, cols]
    for i in lags:
        shifted = tmp.copy()
        shifted_cols = shifted.columns.tolist()
        shifted_cols = [c+'_lag_'+str(i) if c in shift_cols else c 
                        for c in shifted_cols]
        shifted.columns = shifted_cols
        shifted['month'] += i
        shifted.drop_duplicates(inplace=True)
        df = pd.merge(df, shifted, on=merge_cols, how='left')
    if fillna_value is not None:
        df.fillna(fillna_value, inplace=True)
    return df    

In [3]:
def clip_rmse(preds, dtrain):
    y_test = np.array(dtrain.get_label())
    preds = np.array(preds)
    y_test = np.maximum(np.minimum(y_test, 20), 0)
    preds = np.maximum(np.minimum(preds, 20), 0)
    #preds = np.array(list(map(lambda x: min(20, max(x, 0)), list(preds))))
    #y_test = np.array(list(map(lambda x: min(20, max(x, 0)), list(y_test))))
    rmse = np.sqrt(mean_squared_error(preds,y_test))
    return 'clip-rmse', rmse

In [4]:
def cv(x_train, y_train, param, n_repetition, n_split, n_tree, verbose, random_state):
    '''Repeated CV'''
    
    cv_results = {}
    clf = {}
    running_time = {}

    np.random.seed(random_state)

    for m in range(n_repetition):
        # Train and valuation sets split
        skf = StratifiedKFold(n_splits=n_split, random_state=np.random.randint(10**6), shuffle=True)

        for n, (train_index, val_index) in enumerate(skf.split(x_train, y_train)):

            start_time = time.time()

            # Construct DMatrix
            dtrain = xgb.DMatrix(x_train.iloc[train_index], label=y_train.iloc[train_index])
            dval = xgb.DMatrix(x_train.iloc[val_index], label=y_train.iloc[val_index])

            # Placeholder for evals_results
            cv_results[m, n] = {}

            param['seed'] = np.random.randint(10**6)
            clf[m, n] = xgb.train(param, dtrain,num_boost_round=n_tree, 
                                  evals=[(dtrain, 'train'), (dval, 'val')], 
                                  feval=clip_rmse, maximize=False, early_stopping_rounds=None, 
                                  evals_result=cv_results[m, n], verbose_eval=verbose)

            running_time[m, n] = time.time() - start_time

            print('Repeat {}, split {}, val score = {:.3f}, running time = {:.3f} min.'.format(m, n, 
                cv_results[m, n]['val']['clip-rmse'][-1], running_time[m, n]/60))

    cv_results_final = {}
    for m in range(n_repetition):
        for n in range(n_split):
            cv_results_final['train', m, n] = cv_results[m, n]['train']['clip-rmse']
            cv_results_final['val', m, n] = cv_results[m, n]['val']['clip-rmse']

    df = pd.DataFrame(cv_results_final)
    df.index.name = 'iteration'
    df.columns.names = ['dataset', 'repetition', 'cv_split']

    print('Val mean = {:.3f}, std = {:.3f}'.format(df['val'].iloc[-1].mean(), df['val'].iloc[-1].std()))
    
    return df, clf, running_time

In [5]:
def feature_combination(x, feature_list, function_dict, column_name, merge=False):
    '''Combination of new features'''
    tmp = x.groupby(feature_list).agg(function_dict)
    tmp.columns = column_name
    if merge:
        x = x.merge(tmp, on=feature_list, how='left')
        return x, tmp
    else:
        return tmp

### Load and Preprocess Data

In [6]:
# Load data
train = pd.read_csv('all/sales_train.csv.gz')
test = pd.read_csv('all/test.csv.gz')
shop = pd.read_csv('all/shops-translated.csv')
item = pd.read_csv('all/item_category.csv')

test.set_index('ID', inplace=True)
item.drop(['item_name_translated'], axis=1, inplace=True)
shop.drop(['Name'], axis=1, inplace=True)

le = LabelEncoder()
item['item_cat1'] = le.fit_transform(item['item_cat1'].astype(str))
item['item_cat2'] = le.fit_transform(item['item_cat2'].astype(str))
shop['City'] = le.fit_transform(shop['City'])
shop['Type'] = le.fit_transform(shop['Type'])

Remove outliers

In [7]:
train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

Fix shop names and ids

In [8]:
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

Monthly sales for all the samples

In [9]:
x = pd.DataFrame(train.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum())

x.reset_index(inplace=True)

Training set only contains sold samples, need to extend to all samples.

There are two ways of extending:
1. overall product between elements in (month, shop_id, item_id)
2. in each month, the product between elements in (shop_id, item_id)

The first one increases the number of rows by 23.5 times, the second one 6 times

In [10]:
shop_count = {}
item_count = {}
product_count = {}
sample_count = {}
ratio = {}

for n in x.date_block_num.unique():
    shop_count[n] = len(x.loc[x.date_block_num==n, 'shop_id'].unique())
    item_count[n] = len(x.loc[x.date_block_num==n, 'item_id'].unique())
    sample_count[n] = len(x.loc[x.date_block_num==n, :])
    product_count[n] = shop_count[n]*item_count[n]
    ratio[n] = product_count[n]/sample_count[n]
#     print('product count is {}, sample count is {}, ratio is {:.3f}'.format(product_count[n], 
#                                                                             sample_count[n], 
#                                                                             product_count[n]/sample_count[n]))
    
print('overall product count is {}, sample count is {}, ratio is {:.3f}'.format(
    len(x.shop_id.unique())*len(x.item_id.unique())*34, 
    x.shape[0], 34*len(x.shop_id.unique())*len(x.item_id.unique())/x.shape[0]))
print('monthly product count sum is {}, monthly sample count is {}, ratio is {:.3f}'.format(
    sum(product_count.values()), sum(sample_count.values()), sum(product_count.values())/sum(sample_count.values())))

overall product count is 42260028, sample count is 1609123, ratio is 26.263
monthly product count sum is 10913804, monthly sample count is 1609123, ratio is 6.782


In [11]:
tmp = []
for n in range(34):
    shop_list = x.loc[x.date_block_num==n, 'shop_id'].unique()
    item_list = x.loc[x.date_block_num==n, 'item_id'].unique()
    idx = pd.MultiIndex.from_product([[n], shop_list, item_list], names=['date_block_num', 'shop_id', 'item_id'])
    df_tmp = pd.DataFrame(np.zeros((len(idx),2)), index=idx)
    tmp.append(df_tmp)
tmp = pd.concat(tmp, sort=False)
tmp.reset_index(inplace=True, drop=False)
tmp.drop([0, 1], axis=1, inplace=True)
x = tmp.merge(x, on=['date_block_num', 'shop_id', 'item_id'], how='outer').fillna(0.0)
test['date_block_num'] = 34
x = pd.concat((x, test), sort=False).fillna(0.0)

Add shop/item features

In [12]:
x = pd.merge(x, shop, on='shop_id', how='left')
x = pd.merge(x, item, on='item_id', how='left')

Change column names

In [13]:
x.columns = ['month', 'shop_id', 'item_id', 'sales_month', 
             'City', 'Type', 'item_cat1', 'item_cat2']

In [14]:
tmp = x.pivot_table(index=['item_id'], columns='month', values='sales_month', aggfunc=np.sum)
tmp = tmp.replace(0.0, np.nan)

for n in range(tmp.shape[1]):
    idx = np.where(~np.isnan(tmp.iloc[:, n]))[0]
    tmp.iloc[idx, n] = n

tmp0 = pd.DataFrame(tmp.min(axis=1))

tmp0.reset_index(inplace=True)

tmp0.columns = ['item_id', 'item_to_first_sold_month']

x = x.merge(tmp0, on='item_id', how='left')
x['item_to_first_sold_month'] = x['month'] - x['item_to_first_sold_month']

x.loc[x.item_to_first_sold_month<=0.0, 'item_to_first_sold_month'] = np.nan

In [16]:
tmp = x.pivot_table(index=['shop_id', 'item_id'], columns='month', values='sales_month', aggfunc=np.sum)
tmp = tmp.replace(0.0, np.nan)

for n in range(tmp.shape[1]):
    idx = np.where(~np.isnan(tmp.iloc[:, n]))[0]
    tmp.iloc[idx, n] = n

tmp0 = pd.DataFrame(tmp.min(axis=1))
tmp0.reset_index(inplace=True)
tmp0.columns = ['shop_id', 'item_id', 'shop_item_to_first_sold_month']

x = x.merge(tmp0, on=['shop_id', 'item_id'], how='left')
x['shop_item_to_first_sold_month'] = x['month'] - x['shop_item_to_first_sold_month']

x.loc[x.item_to_first_sold_month<=0.0, 'shop_item_to_first_sold_month'] = np.nan

In [21]:
tmp0[(tmp0.shop_id==59) & (tmp0.item_id==14794)]

,shop_id,item_id,shop_item_to_first_sold_month
1121520,59,14794,NaN


In [22]:
x.loc[(x.month==1)&(x.shop_id==59)&(x.item_id==14794)]

,month,shop_id,item_id,sales_month,City,Type,item_cat1,item_cat2,item_to_first_sold_month,shop_item_to_first_sold_month
737292,1,59,14794,0.0,27,1,3,9,1.0,NaN


In [23]:
x.sample(50)

,month,shop_id,item_id,sales_month,City,Type,item_cat1,item_cat2,item_to_first_sold_month,shop_item_to_first_sold_month
1705509,4,32,2500,0.0,8,4,16,25,-1.0,-1.0
10444220,31,57,6307,0.0,26,3,12,16,8.0,8.0
4392123,11,47,12349,0.0,18,2,3,23,11.0,11.0
7300634,20,33,7306,0.0,9,5,12,16,12.0,NaN
9906464,29,35,13542,1.0,10,5,10,24,10.0,10.0
3887789,10,25,10018,0.0,8,5,3,23,10.0,8.0
877846,2,22,2772,0.0,8,3,16,52,2.0,-1.0
6788177,18,54,21647,0.0,5,4,3,9,17.0,14.0
2859060,7,37,2175,0.0,11,4,12,16,7.0,NaN
4998642,13,29,10225,0.0,8,4,2,5,13.0,9.0
